# Lab 4: Free Particle Propagator (Assignment 1 Tutorial)

In assignment 1, we will perform the path integral numerically for the harmonic oscillator.
In today's lab, we'll start with a simpler problem: the free particle.

\begin{equation}
L = K = \frac{1}{2}\dot{x}^2
\end{equation}

where $m = \hbar = 1$ in these units.

## Setup

We will use the discrete approximation to the path integral over a time period $T_0 = 2\pi$, where the time step is $\epsilon = \Delta t = T_0/128$.
The electron position is also discretized into $N_D+1$ possible points, $x_0 = -4, x_1, x_2, \ldots, x_{N_D} = +4$, where $N_D=600$.

Let's set these up as constants in our code.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

T0 = 2 * np.pi 
NT = 128 
DELTAT = T0 / NT
BOXSIZE = 8
ND = 600
DELTAX = BOXSIZE / ND
HBAR = 1

## Initial probability amplitude

The initial probability amplitude (sometimes called the wavefunction) of the electron is a Gaussian centered at $x_\mathrm{start}$,

\begin{equation}
  \Psi_0(x) = \left(\frac{\alpha}{\pi}\right)^{1/4}\exp\left(-\frac{\alpha}{2}(x-x_\mathrm{start})^2\right),
\end{equation}

where $\alpha = 2$ and $x_\mathrm{start} = 0$.
The amplitude can be represented as a vector $\psi_0$ with $N_D+1$ components, $\psi_0 = (\Psi_0(x_0), \Psi_0(x_1), \ldots, \Psi_0(x_{N_D}))$.

We will plot this initial amplitude and check it's normalization.

In [ ]:
XSTART = 0

t = np.linspace(0, T0, NT + 1)
x = np.linspace(-BOXSIZE / 2, BOXSIZE / 2, ND + 1)

def func_psi_0(x, x_start):
    alpha = 2
    return (alpha / np.pi) ** (1 / 4) * np.exp(-alpha / 2 * (x - x_start)**2)

psi_0 = func_psi_0(x, XSTART)

print(f"Psi at every 10th point: {psi_0[::10]}")
print(f"Normalization condition:  {DELTAX * sum(psi_0 * psi_0.conjugate())}")

## Exercise 1: Plot the initial probability distribution

In [ ]:

xmin, xmax = -BOXSIZE / 2, BOXSIZE / 2
plt.figure(figsize=(6, 6))
# plot the initial probability distribution here
plt.plot(x, psi_0 * psi_0.transpose(), label=r"$|\psi_0|^2$")
plt.xlabel(r"$x$")
plt.ylabel(r"$|\psi|^2$")
plt.xlim([xmin, xmax])
plt.legend()
plt.show()

## Propagator matrix

From lecture, we have the free particle propagator,

\begin{equation}
\mathcal K(x_b, t_b; x_a, t_a) = \left (\frac{m}{2\pi i \hbar (t_b -t_a)}  \right )^{1/2}\exp\left [\frac{im(x_b - x_a)^2}{2\hbar(t_b-t_a)}\right ].
\end{equation}

In this problem, we can reduce this to

\begin{equation}
\mathcal K_\epsilon(x_j; x_i) = \mathcal K(x_j, \epsilon; x_i, 0) = \left (\frac{m}{2\pi i \hbar \epsilon} \right )^{1/2}\exp\left [\frac{im(x_j - x_i)^2}{2\hbar\epsilon}\right ].
\end{equation}

Here, we use $x_i$ and $x_j$ because $x$ is discretized.

Since $x$ is discretized, we can write this as a matrix,

\begin{equation}
(K_\epsilon)_{i, j} = K_\epsilon(x_j; x_i).
\end{equation}

Note, we can get matrix $K_{t}$ at any time $t$

\begin{equation}
K_{t} = (\Delta x)^{N-1} {K_\epsilon}^N,
\end{equation}

where $t = N \epsilon$.

The assignment asks you to calculate the propagator matrix $\mathcal K_{8\epsilon}$ for a time period $T_0/16 = 8\epsilon$ (8 time steps) built from the elementary propagator matrix $\mathcal K_\epsilon$ for a single $\epsilon=\Delta t=T_0/128$ time step.
You can do this with the formula above, setting $N=8$.

In [ ]:
def func_K(x_j, x_i, dt):
    coefficient = np.sqrt(1 / (2 * np.pi * 1j * HBAR * dt))
    exponent = 1j * (x_j - x_i)**2 / (2 * HBAR * dt)
    return coefficient * np.exp(exponent)

K_dt = np.zeros((ND + 1, ND + 1), dtype=np.complex64)
for i in range(ND + 1):
    for j in range(ND + 1):
        K_dt[i, j] = func_K(x[j], x[i], DELTAT)
    

## Evolve the probability amplitude

Next, we will evolve the probability amplitude (wave function) of the electron by one time step at a time.

We will also compute the probability distribution $|\psi_t(x)|^2$ at each time step.

Note, given $\psi_t(x)$ at a given time, we are able to find the mean position $\langle x \rangle$ by 

\begin{equation}
\langle x\rangle = \int x |\psi_t(x)|^2\ dx,
\end{equation}
but we'll skip that for now.

In [ ]:
psi = [psi_0]

for i in range(1, NT + 1):
    psi_t = DELTAX * np.matmul(K_dt, psi[i-1])
    psi.append(psi_t)

prob = []
for i in range(NT + 1):
    prob.append(np.real(psi[i] * psi[i].conjugate()))


## Exercise 2: Plot the evolution of the probability distribution

Let's plot the evolution of the probability distribution every 2 time steps (i.e. $2\epsilon = T_0/64$) from $t=0$ to $T_0/8 = \pi/4$.

In [ ]:
plt.figure(figsize=(6, 6))
plot_interval = 2
for i in range(0, NT // 8 + 1, plot_interval):
    # plot the probability distribution and label it with the time
    plt.plot(x, prob[i], label=f"t = {t[i]:.2f}")
plt.xlabel(r"$x$")
plt.ylabel(r"$|\psi|^2$")
plt.xlim([xmin, xmax])
plt.legend()
plt.show()

## Changing the initial wave function

It seems like the mean position of the particle doesn't move, but the probability does "disperse" away. Why is that? The initial wave function is a Gaussian centered at $x_\mathrm{start} = 0$ basically representing a particle at rest.
But this has contributions from all possible momenta, so the particle has some probability to be found away from the origin with amplitude given according to the free propagator.

Let's try a different wave function with some definite initial momentum $p=\hbar k_0$,

\begin{equation}
  \Psi_0(x) = \left(\frac{\alpha}{\pi}\right)^{1/4}\exp\left(-\frac{\alpha}{2}(x-x_\mathrm{start})^2 + i k_0 (x-x_\mathrm{start})\right),
\end{equation}

and again $\alpha = 2$ and $x_\mathrm{start} = 0$, and now let's set the wavenumber $k_0 = 1$ . 

In [ ]:
K0 = 1

def func_psi_0(x, x_start):
    alpha = 2
    return (alpha / np.pi) ** (1 / 4) * np.exp(-(alpha / 2) * (x - x_start)**2 + 1j * K0 * (x - x_start))

psi_0 = func_psi_0(x, XSTART)

print(f"Psi at every 10th point: {psi_0[::10]}")
print(f"Normalization condition:  {DELTAX * sum(psi_0 * psi_0.conjugate())}")

In [ ]:
psi = [psi_0]

for i in range(1, NT + 1):
    psi_t = DELTAX * np.matmul(K_dt, psi[i-1])
    psi.append(psi_t)

prob = []
for i in range(NT + 1):
    prob.append(np.real(psi[i] * psi[i].conjugate()))

## Exercise 3: Plot the evolution of the probability distribution

Again, let's plot the evolution of the probability distribution every 2 time steps (i.e. $2\epsilon = T_0/64$) from $t=0$ to $T_0/8 = \pi/4$.

In [ ]:
plt.figure(figsize=(6, 6))
for i in range(0, NT // 8 + 1, plot_interval):
    # plot the probability distribution and label it with the time
    plt.plot(x, prob[i], label=f"t = {t[i]:.2f}")
plt.xlabel(r"$x$")
plt.ylabel(r"$|\psi|^2$")
plt.xlim([xmin, xmax])
plt.legend()
plt.show()